In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import sys
import os

In [ ]:
plt.style.use('ggplot')
plt.rcParams['figure.dpi'] = 150
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['legend.fontsize'] = 6
plt.rcParams['axes.titlesize'] = 7
plt.rcParams['axes.labelsize'] = 8
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8

In [ ]:
plot_water_balance = True
scratch_path = os.environ.get('SCRATCH')

In [ ]:
cases = ['', '']
cases

In [ ]:
for i, case in enumerate(cases):
    df = pd.read_csv(f'{case}/pflotran_surface_total_concentration.csv', comment='#').merge(pd.read_csv(f'{case}/pflotran_subsurface_total_concentration.csv', comment='#'))
    if plot_water_balance:
        w = pd.read_csv(f'{case}/water_balance_computational_domain.csv', comment='#')
    if i == 0:
        print(df.columns)
        if plot_water_balance:
            print(w.columns)
    print('start =', df.iloc[0, 0]/86400, 'end =', df.iloc[-1, 0]/86400, 'nrow =', len(df), 'ncol =', len(df.columns))
    if i == 0:
        t0a = df.iloc[0, 0]
        if plot_water_balance:
            t0b = w.iloc[0, 0]
        fig, axs = plt.subplots(len(df.columns)//4+1, 4, figsize=(12,9), sharex=True)
    df.iloc[:, 0] -= t0a
    df.iloc[:, 0] /= 86400
    df.iloc[0, 1:] = df.iloc[1, 1:]
    df.rename(columns={"time [s]": "time [d]"}, inplace=True)
    if plot_water_balance:
        w.iloc[:, 0] -= t0b
        # w.iloc[:, 0] /= 365
        w.iloc[0, 1:] = w.iloc[1, 1:]
    for j, ax in enumerate(axs.flat):
        if j > len(df.columns):
            break
        if plot_water_balance:
            if j == len(df.columns) - 1:
                ax.plot(w.iloc[:,0], w.iloc[:,4]*1000, color='k')
                ax.set_title('rain [mm]', fontweight='bold')
            elif j == len(df.columns):
                ax.plot(w.iloc[:,0], w.iloc[:,5]*1000, color='k')
                ax.plot(w.iloc[:,0], w.iloc[:,9]*1000, color='k', ls='--', label='snowmelt')
                ax.set_title('snow [mm]', fontweight='bold')
                if i == 0:
                    ax.legend(loc='upper left')
            else:
                ax.semilogy(df.iloc[:, 0], df.iloc[:, j+1]*55000, label=df.columns[j+1]) # plot semilogy
                # if i != len(sys.argv)-2:
                     # ax.scatter(df.iloc[-1, 0], df.iloc[-1, j+1], color='k', s=3)
                ax.set_title(str(df.columns[j+1]), fontweight='bold')
            ax.grid(which='both', axis='both', linestyle='--', linewidth=0.5)
            if j > len(axs.flat)-5:
                ax.set_xlabel(df.columns[0])
        else:
            ax.plot(df.iloc[:, 0], df.iloc[:, j+1], label=df.columns[j+1])
            ax.set_title(str(df.columns[j+1]), fontweight='bold')
plt.tight_layout()
print()
plt.show()